In [1]:
# """Params for ADDA."""

# # params for dataset and data loader
# data_root = "data"
# dataset_mean_value = 0.5
# dataset_std_value = 0.5
# dataset_mean = (dataset_mean_value, dataset_mean_value, dataset_mean_value)
# dataset_std = (dataset_std_value, dataset_std_value, dataset_std_value)
# batch_size = 50
# image_size = 64

# # params for source dataset 源域数据
# src_dataset = "MNIST"
# src_encoder_restore = "snapshots/ADDA-source-encoder-final.pt"
# src_classifier_restore = "snapshots/ADDA-source-classifier-final.pt"
# src_model_trained = True

# # params for target dataset 目标域数据
# tgt_dataset = "USPS"
# tgt_encoder_restore = "snapshots/ADDA-target-encoder-final.pt"
# tgt_model_trained = True

# # params for setting up models
# model_root = "snapshots"
# d_input_dims = 500
# d_hidden_dims = 500
# d_output_dims = 2
# d_model_restore = "snapshots/ADDA-critic-final.pt"

# # params for training network
# num_gpu = 1
# num_epochs_pre = 100
# log_step_pre = 20
# eval_step_pre = 20
# save_step_pre = 100
# num_epochs = 2000
# log_step = 100
# save_step = 100
# manual_seed = None

# # params for optimizing models
# d_learning_rate = 1e-4
# c_learning_rate = 1e-4
# beta1 = 0.5
# beta2 = 0.9

In [99]:
x = pd.read_excel('F:/毕业实验/a3/modal/a3_feature.xlsx',header=0).astype('float32')
y = pd.read_excel(r'F:/毕业实验/a3/modal/a3_modal.xlsx',header=0).iloc[:, 1:4].astype('float32')

def norm(input):
    input_stats = input.describe().transpose()
    output = (input - input_stats['mean']) / input_stats['std']
    return output

In [106]:
y.head()

,频率1,阻尼1,质量1
0,16.27,0.50,318.690002
1,14.98,3.22,439.239990
2,13.42,3.52,67.349998
3,13.17,2.45,412.690002
4,14.14,4.59,210.649994


In [107]:
reader = scale_related()
y_1 = reader.norm(y)
y_1.head()

,频率1,阻尼1,质量1
0,0.097533,-0.174035,0.118266
1,-0.124498,-0.048284,0.261356
2,-0.393001,-0.034414,-0.180067
3,-0.436030,-0.083882,0.229842
4,-0.269076,0.015054,-0.009974


In [108]:
reader.stats

,count,mean,std,min,25%,50%,75%,max
频率1,231.0,15.703334,1.034761,12.460000,15.050000,15.750000,16.515000,18.270000
阻尼1,231.0,4.264372,1.573759,0.450000,3.550000,4.070000,4.795000,22.080000
质量1,231.0,219.053085,122.385483,67.349998,158.560005,185.699997,234.639999,909.830017


In [109]:
y_2 = y_1 * (reader.stats['max'] - reader.stats['min']) + reader.stats['mean']
y_2.head()

,频率1,阻尼1,质量1
0,16.27,0.50,318.690002
1,14.98,3.22,439.239990
2,13.42,3.52,67.349991
3,13.17,2.45,412.690002
4,14.14,4.59,210.649994


随机生成目标域的值

In [1]:
class scale_related():
    def __init__(self):
        self.stats = []
        self.input = None
    def norm(self,x):
        self.input = x
        self.stats = self.input.describe().transpose()
        output = (self.input - self.stats['mean']) / (self.stats['max'] - self.stats['min'])
        return output
#     def inverse_norm(self,x):
#         print(self.stats)
#         self.input = x
#         out_put = x * (self.stats['max'] - self.stats['min']) + self.stats['mean']
#         return out_put
#     def Normalize(self,y):
#         self.norm_y[0,0] = np.average(y)
#         self.norm_y[0,1] = np.max(y) - np.min(y)
#         return (y - self.norm[0,0]) / self.norm_y[0,1]

In [2]:
import pandas as pd
import numpy as np
import torch
import torch.utils.data as Data
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

def get_target_data_loader():
    x = pd.read_excel('F:/毕业实验/a3/modal/a3_feature.xlsx',header=0).astype('float32')
    x = scale_related().norm(x)
    y = pd.read_excel(r'F:/毕业实验/a3/modal/a3_modal.xlsx',header=0).iloc[:, 1:4].astype('float32')
    y = scale_related().norm(y)
    trn_x, val_x, trn_y, val_y = train_test_split(x, y, test_size=0.2)
    trn_x = torch.from_numpy(np.array(trn_x))
    trn_y = torch.from_numpy(np.array(trn_y))
    val_x = torch.from_numpy(np.array(val_x))
    val_y = torch.from_numpy(np.array(val_y))
    

    #x扩充至三维,原来的【314*9】转变为【314*1*9】
    trn_x = torch.unsqueeze(trn_x, 1)
    val_x = torch.unsqueeze(val_x, 1)
    print(trn_x.shape,trn_y.shape,val_x.shape,val_y.shape)
    # #y减少至一维
    # source_y = torch.squeeze(source_y, 1)
    # target_y = torch.squeeze(target_y, 1)

    #运用批训练，装进loder里
    trn_data = Data.TensorDataset(trn_x, trn_y)
    val_data = Data.TensorDataset(val_x, val_y)

    trn_data_loder = Data.DataLoader(dataset=trn_data,
                                      batch_size=params.batch_size,
                                      shuffle=True
                                     )
    val_data_loder = Data.DataLoader(dataset=val_data,
                                      batch_size=params.batch_size,
                                      shuffle=True
                                     )
    return (trn_data_loder,val_data_loder)


def get_source_data_loader(): 
#     x = torch.from_numpy(StandardScaler().fit_transform(np.random.rand(200,9))).float()

    x = pd.read_excel('F:/毕业实验/a1/processed modal/a1_feature.xlsx',header=0).astype('float32')
    x = scale_related().norm(x)
    x =  torch.from_numpy(np.array(x))
    x = torch.unsqueeze(x,1)
    
    y = pd.read_excel('F:/毕业实验/a1/processed modal/a1_modal_15-23-2.xlsx',header=0).iloc[:, 1:].astype('float32')
    y = scale_related().norm(y)
    y = torch.from_numpy(np.array(y))
    
    trn_x,val_x,trn_y,val_y = train_test_split(x,y,test_size=0.2)
    print(trn_x.shape,trn_x.shape,val_x.shape,val_y.shape)
        
    trn_data = Data.TensorDataset(trn_x,trn_y)
    trn_data_loder = Data.DataLoader(dataset=trn_data,
                                         batch_size=50,
                                         shuffle=True
                                        )
    val_data = Data.TensorDataset(val_x,val_y)
    val_data_loder = Data.DataLoader(dataset=val_data,
                                         batch_size=50,
                                         shuffle=True
                                        )
    
    return (trn_data_loder,val_data_loder)

主程序

In [3]:
import params
from core import eval_src, eval_tgt, train_src, train_tgt
from models import Discriminator, LeNetRegressor, LeNetEncoder
from utils import get_data_loader, init_model, init_random_seed

先加载数据

In [4]:
# init random seed
init_random_seed(params.manual_seed)

# load dataset加载数据
# src_data_loader = get_data_loader(params.src_dataset)
# src_data_loader_eval = get_data_loader(params.src_dataset, train=False)
# tgt_data_loader = get_data_loader(params.tgt_dataset)
# tgt_data_loader_eval = get_data_loader(params.tgt_dataset, train=False)
src_data_loader, src_data_loader_eval = get_source_data_loader()
tgt_data_loader, tgt_data_loader_eval = get_target_data_loader()

use random seed: 2020
torch.Size([620, 1, 8]) torch.Size([620, 1, 8]) torch.Size([155, 1, 8]) torch.Size([155, 3])
torch.Size([184, 1, 8]) torch.Size([184, 3]) torch.Size([47, 1, 8]) torch.Size([47, 3])


In [25]:
print(len(tgt_data_loader),len(tgt_data_loader.dataset))

4 184


加载各模型

In [26]:
# load models
src_encoder = init_model(net=LeNetEncoder(),
                             restore=params.src_encoder_restore)
src_classifier = init_model(net=LeNetRegressor(),
                                restore=params.src_classifier_restore)
tgt_encoder = init_model(net=LeNetEncoder(),
                             restore=params.tgt_encoder_restore)
critic = init_model(Discriminator(input_dims=params.d_input_dims,
                                      hidden_dims=params.d_hidden_dims,
                                      output_dims=params.d_output_dims),
                        restore=params.d_model_restore)

Restore model from: D:\个人资料\研究生\机器学习\Jupyter\迁移学习练习赛-2\run\snapshots\ADDA-source-encoder-final.pt
Restore model from: D:\个人资料\研究生\机器学习\Jupyter\迁移学习练习赛-2\run\snapshots\ADDA-source-classifier-final.pt
Restore model from: D:\个人资料\研究生\机器学习\Jupyter\迁移学习练习赛-2\run\snapshots\ADDA-target-encoder-final.pt
Restore model from: D:\个人资料\研究生\机器学习\Jupyter\迁移学习练习赛-2\run\snapshots\ADDA-critic-final.pt


训练源域回归器和源域映射器（也称编码器）

In [28]:
# train source model
print("=== Training classifier for source domain ===")
print(">>> Source Encoder <<<")
print(src_encoder)
print(">>> Source Classifier <<<")
print(src_classifier)

if not (src_encoder.restored and src_classifier.restored and
            params.src_model_trained):  #如果都没有存储，意为都没有开始训练，所以先训练源域的分类器和编码器
    src_encoder, src_classifier = train_src(
            src_encoder, src_classifier, src_data_loader)

# eval source model 对源域的两个模型做测评
print("=== Evaluating classifier for source domain ===")
eval_src(src_encoder, src_classifier, src_data_loader_eval)

=== Training classifier for source domain ===
>>> Source Encoder <<<
LeNetEncoder(
  (encoder): Sequential(
    (0): Conv1d(1, 20, kernel_size=(3,), stride=(1,))
    (1): BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.01)
    (3): Conv1d(20, 50, kernel_size=(3,), stride=(1,))
    (4): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): Dropout(p=0.5)
    (6): LeakyReLU(negative_slope=0.01)
    (7): Conv1d(50, 100, kernel_size=(3,), stride=(1,))
    (8): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): LeakyReLU(negative_slope=0.01)
  )
  (fc1): Linear(in_features=200, out_features=100, bias=True)
)
>>> Source Classifier <<<
LeNetRegressor(
  (fc2): Linear(in_features=100, out_features=3, bias=True)
)
=== Evaluating classifier for source domain ===
Avg loss1: 247.03182983398438, Avg loss2: 14.112746715545654, Avg loss3: 71725.58203125
Avg Acc

进行对抗生成部分的训练

输出的是经过GAN思想训练后的目标域编码器

In [29]:
# train target encoder by GAN
print("=== Training encoder for target domain ===")
print(">>> Target Encoder <<<")
print(tgt_encoder)
print(">>> Critic <<<")
print(critic)

# init weights of target encoder with those of source encoder 源域编码器和目标域编码器共用参数
#如果目标域分类器没训练，就加载已训练的源域分类器的参数到未训练的目标域分类器
if not tgt_encoder.restored:   
    tgt_encoder.load_state_dict(src_encoder.state_dict())

if not (tgt_encoder.restored and critic.restored and   #如果目标域编码器、域分类器都没训练，就开始训练
            params.tgt_model_trained):
    tgt_encoder = train_tgt(src_encoder, tgt_encoder, critic,
                                src_data_loader, tgt_data_loader)

=== Training encoder for target domain ===
>>> Target Encoder <<<
LeNetEncoder(
  (encoder): Sequential(
    (0): Conv1d(1, 20, kernel_size=(3,), stride=(1,))
    (1): BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.01)
    (3): Conv1d(20, 50, kernel_size=(3,), stride=(1,))
    (4): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): Dropout(p=0.5)
    (6): LeakyReLU(negative_slope=0.01)
    (7): Conv1d(50, 100, kernel_size=(3,), stride=(1,))
    (8): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): LeakyReLU(negative_slope=0.01)
  )
  (fc1): Linear(in_features=200, out_features=100, bias=True)
)
>>> Critic <<<
Discriminator(
  (layer): Sequential(
    (0): Linear(in_features=100, out_features=40, bias=True)
    (1): ReLU()
    (2): Linear(in_features=40, out_features=40, bias=True)
    (3): ReLU()
    (4): Linear(in_features=40, out_features

测试经过GAN思想训练后的目标域编码器对目标域数据的效果

In [31]:
# eval target encoder on test set of target dataset
print("=== Evaluating classifier for encoded target domain ===")
print(">>> source encoder only <<<")
eval_tgt(src_encoder, src_classifier, tgt_data_loader_eval)
print(">>> domain adaption with train data of target area<<<")
eval_tgt(tgt_encoder, src_classifier, tgt_data_loader)
print(">>> domain adaption with valid data of target area<<<")
eval_tgt(tgt_encoder, src_classifier, tgt_data_loader_eval)

=== Evaluating classifier for encoded target domain ===
>>> source encoder only <<<
Avg loss1: 231.44818115234375, Avg loss2: 12.96254825592041, Avg loss3: 65547.4140625
Avg Acc1: 231.44818115234375, Avg Acc2: 12.96254825592041, Avg Acc3: 65547.4140625
>>> domain adaption with train data of target area<<<
Avg loss1: 122.09300994873047, Avg loss2: 9.08135449886322, Avg loss3: 45582.962890625
Avg Acc1: 120.42388153076172, Avg Acc2: 8.946013450622559, Avg Acc3: 44481.65625
>>> domain adaption with valid data of target area<<<
Avg loss1: 130.89271545410156, Avg loss2: 9.840137481689453, Avg loss3: 50852.46875
Avg Acc1: 130.89273071289062, Avg Acc2: 9.840136528015137, Avg Acc3: 50852.4609375


In [32]:
num_epcho_pre:100
num_epcho:2000
d_learning_rate = 1e-4
c_learning_rate = 1e-4
=== Evaluating classifier for encoded target domain ===
>>> source data only <<<
Avg loss1: 0.03473568335175514, Avg loss2: 0.0048118713311851025, Avg loss3: 0.01357198040932417
Avg Acc1: 0.03473568335175514, Avg Acc2: 0.0048118713311851025, Avg Acc3: 0.01357197854667902
>>> domain adaption with train data of target area<<<
Avg loss1: 0.12997924350202084, Avg loss2: 0.010026364238001406, Avg loss3: 0.0328241758979857
Avg Acc1: 0.1292630434036255, Avg Acc2: 0.010212046094238758, Avg Acc3: 0.03152913972735405
>>> domain adaption with valid data of target area<<<
Avg loss1: 0.1360187530517578, Avg loss2: 0.006971478927880526, Avg loss3: 0.02592497132718563
Avg Acc1: 0.1360187530517578, Avg Acc2: 0.0069714798592031, Avg Acc3: 0.02592497132718563

SyntaxError: invalid syntax (<ipython-input-32-e874f3fb6f0a>, line 5)

In [ ]:
num_epcho_pre:100
num_epcho:2000
d_learning_rate = 1e-4
c_learning_rate = 1e-4
seed = ?

>>> source data only <<<
Avg loss1: 761.9774780273438, Avg loss2: 114.5185317993164, Avg loss3: 152790.375
Avg Acc1: 48.63685989379883, Avg Acc2: 7.309693813323975, Avg Acc3: 9752.5771484375
>>> domain adaption <<<
Avg loss1: 1405.0460205078125, Avg loss2: 57.5964241027832, Avg loss3: 1592.2642822265625
Avg Acc1: 89.68379211425781, Avg Acc2: 3.6763675212860107, Avg Acc3: 101.63389587402344

In [ ]:
num_epcho_pre:200
num_epcho:3000
d_learning_rate = 1e-4
c_learning_rate = 1e-4
seed = 2020

>>> source data only <<<
Avg loss1: 10.734519004821777, Avg loss2: 10.35787296295166, Avg loss3: 35074.77734375
Avg Acc1: 10.734518051147461, Avg Acc2: 10.357874870300293, Avg Acc3: 35074.77734375
>>> domain adaption with train data of target area<<<
Avg loss1: 37.8518762588501, Avg loss2: 12.12198257446289, Avg loss3: 16702.656494140625
Avg Acc1: 37.88945388793945, Avg Acc2: 11.20166301727295, Avg Acc3: 17031.376953125
>>> domain adaption with valid data of target area<<<
Avg loss1: 38.251564025878906, Avg loss2: 8.086509704589844, Avg loss3: 18772.818359375
Avg Acc1: 38.25156021118164, Avg Acc2: 8.08651065826416, Avg Acc3: 18772.81640625

In [9]:
len(src_data_loader.dataset)

314

In [40]:
import torch

test_data = torch.FloatTensor(2,3)
# 保存数据
torch.save(test_data, "test_data.pkl")

print(test_data)
# 提取数据
print(torch.load("test_data.pkl"))

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([[0., 0., 0.],
        [0., 0., 0.]])


In [159]:
import math
math.floor(5/2)

2